### Práctica de Formatos de datos: XML

La __EMT__ dispone de un portal de datos abiertos que se puede encontrar en la página http://opendata.emtmadrid.es/Home. El objetivo del mismo es ofrecer diferentes tipos de datos sobre la actividad de los autobuses de la EMT. El acceso a los datos se realiza a través de una API de servicios web. 

Para poder utilizar la API de servicios, en primre lugar hay que registrarse, lo cual puede hacerse en la página http://opendata.emtmadrid.es/Formulario. El registro devuelve como resultado en un mensaje electrónico, dos valores:

* Id de cliente: Identificador de cliente

* Pass Key: Password

A continuación, hay que elegir el servicio que se quiere utilizar. Hay 4 servicios definidos: BUS, GEO, MEDIA, INFOPARKING y BICIMAD. Cada servicio tiene asociado un conjunto de métodos que al invocarlos, devuelven un resultado. Por ejemplo, el servicio BUS dispone del servicio __"GetRouteLines"__ que obtiene el itinerario de una línea (o varias líneas separadas por el carácter pipe(|)), con los vértices para construir las rectas del recorrido y las coordenadas UTM de los ejes viales y los códigos de parada. Si se quiere invocar desde Python, se puede hacer usando el siguiente código:

# En esta práctica se va a trabajar con varios métodos:

* Método __'GetStreet'__ del servicio GEO. Este método obtiene una lista de emplazamientos EMT coincidentes con una localización. Cada emplazamiento está compuesto por una lista de paradas EMT situadas dentro de un radio predefinido con todos sus atributos, así como las líneas EMT que pasan por cada parada de la lista. En el documento __Servicios_EMT.pdf__ adjunto a esta práctica, se explica con detalle los parámetros de entrada y el resultado que devuelve este método. El método se puede invocar indicando al menos el nombre de la calle y el número:

El método devuelve un documento XML. Entre los elementos del documento, se encuentran las coordenadas X e Y del lugar buscado. En la búsqueda del ejemplo, la calle búscada tiene como coordenadas UTM X e Y:

* Método __'GetStreetRoute'__ del servicio MEDIA. Este método obtiene hasta tres rutas óptimas para ir de un sitio a otro a pie y en autobús. En el documento __Servicios_EMT.pdf__ adjunto a esta práctica, se explica con detalle los parámetros de entrada y el resultado que devuelve este método. El método se puede invocar indicando los siguientes parámetros

  * idClient: Código de cliente autorizado para la operación y suministrado por EMT.

  * passKey: Código de clave asociado al cliente.

  * coordinateXFrom: Campo para identificar la posición "x" del origen de la ruta a consultar. 
  
  * coordinateYFrom: Campo para identificar la posición "y" del origen de la ruta a consultar.

  * coordinateXTo: Campo para identificar la posición "x" del destino de la ruta a consultar. 
  
  * coordinateYTo: Campo para identificar la posición "y" del destino de la ruta a consultar. 

  * criteriaSelection: Campo que indica el criterio de la búsqueda. Los valores son:
  
       * 11 - Mínimo tiempo de trayecto
    
       * 13 - Mínimos trasbordos
    
       * 14 - Mínimo recorrido a pie

El método devuelve un documento XML que contiene información sobre las rutas óptimas (se explica con detalle en el documento __Servicios_EMT.pdf__). En particular muestra la siguiente información:

* El elemento __DescriptionRouteData__ donde aparece la siguiente información:

  * DescriptionDate: Fecha de la ruta.
  * DescriptionInitTime: Descripción de hora de inicio de la ruta.
  * DescriptionEstimateTimeArrival: Descripción de la hora estimada de llegada.
  * Transfers: Trasbordos.
  * LongJourney: Duración del viaje.

* El elemento __ListSectionRoute__ aparece una lista de subelementos __Section__.Cada uno de ellos, describe parcialmente una parte de la ruta. En un caso ideal, aparecerá un subelemento de __Section__ de tipo __WalkingLeg__  que describe el inicio de la ruta andando, a continuación un conjunto de subelementos de __Section__ de tipo __BusLeg__ que describen cada uno de ellos la ruta en una línea de autobus que forma parte de la ruta buscada, y por último otro subelemento de __Section__ de tipo __WalkingLeg__  que describe el final de la ruta andando.

* El elemento __POI__ que muestra información sobre puntos de interés que se encuentran a lo largo de la ruta.

Se pide:

* [1 punto] Crear una función que solicite al usuario dos calles de Madrid, que actúen de origen y destino de una ruta

* [2 puntos]Crear una función que dada una calle de Madrid, devuelva las coordenadas X e Y de la calle.

* [4 puntos]Crear una función que dadas las coordenadas X e Y de un origen y un destino dentro de Madrid, obtenga la ruta más óptima para ir del origen al destino usando autobuses de la EMT. La función deberá imprimir por pantalla, la siguiente información:

 * Fecha de la ruta.
 
 * Hora de inicio de la ruta.
 
 * Hora estimada de llegada
 
 * Número de trasbordos.
 
 * Duración del viaje.
 
 * Descripción textual de la ruta.
 
 Usando el ejemplo anterior debería mostrar:
 
 * Fecha de la ruta: 26/10/2017
 
 * Hora de inicio de la ruta: 03:55
 
 * Hora estimada de llegada: 05:28
 
 * Número de trasbordos: 1
 
 * Duración del viaje: 93
 
 * Descripción textual de la ruta:
 
     * Caminar 42' hasta parada 3125 - EMILIO VARGAS-ARTURO SORIA, linea N4
     
     * 28' en autobús (línea N4) hasta parada 449 - SERRANO-ORTEGA Y GASSET
     
     * Desde parada 449 caminar 5' hasta parada 61 - Castellana-Ministerio Interior, linea N25
 
     * 15' en autobús (línea N25) hasta parada 3691 - SOL-SEVILLA

     * Desde parada 3691 caminar 3'

* [1 punto]Crear una función que solicite al usuarios los nombres de 2 calles de Madrid y muestre la información referente a la ruta más óptima para llegar de una calle a otra usando autobuses de la EMT.

* [2 puntos]Crear una función que solicite al usuarios los nombres de 2 calles de Madrid y muestre un listado de todos los puntos de interés que hay dentro de la ruta más óptima para llegar de una calle a otra usando autobuses de la EMT. A continuación, se le pedirá al usuario que introduzca el nombre de uno de los puntos de interés mostrado, y como resultado se mostrará por pantalla los siguientes datos del punto de interés:

  * Nombre del punto de interés.
  
  * Dirección del punto de interés.
  
  * Teléfono del punto de interés.

In [12]:
from xml.etree import ElementTree
from xml.etree.ElementTree import Element
import requests


#EJERCICIO 1
def leerCalles():
    xyOrigen=[]
    xyDestino=[]
    calleOrigen = input("Introduzca la calle origen:")
    calleDestino = input("Introduzca la calle destino:")
    
    return calleOrigen,calleDestino

#EJERCICIO 2
def coordenadas(calle):
    datos = {
        'idClient':'WEB.SERV.segonz09@ucm.es',
        'PassKey':'973F19F0-B74A-407C-82E1-37D96CC9940B',
        'description': calle,
        'streetNumber':'',
        'Radius':'',
        'Stops':'',
        'statistics':'',
        'cultureInfo':''
    }

    url = 'https://servicios.emtmadrid.es:8443/geo/servicegeo.asmx/GetStreet'
    response = requests.post(url, data=datos)
    ficheroXML = open("infoCalle.xml", 'w')
    ficheroXML.write(response.text)
    ficheroXML.close()

    f= open("infoCalle.xml", "rt")
    arbol=ElementTree.parse(f)
    nodo = arbol.find('Site')
    coorX = nodo.find('CoordinateX')
    coorY = nodo.find('CoordinateY')
    xy=[]
    xy.append(coorX.text)
    xy.append(coorY.text)
        
    return xy

#EJERCICIO 3
def caminoOptimo(xyOrigen,xyDestino):
    datos = {
        'idClient':'WEB.SERV.segonz09@ucm.es',
        'PassKey':'973F19F0-B74A-407C-82E1-37D96CC9940B',
        'statistics':'',
        'cultureInfo':'',
        'coordinateXFrom': xyOrigen[0],
        'coordinateYFrom':xyOrigen[1],
        'originName':'',
        'coordinateXTo': xyDestino[0],
        'coordinateYTo':xyDestino[1],
        'destinationName':'',
        'criteriaSelection':'13',
        'day':'',
        'month':'',
        'year':'',
        'hour':'',
        'minute':'',
        'GenerarAudio':''
    }

    url = 'https://servicios.emtmadrid.es:8443/servicemedia/servicemedia.asmx/GetStreetRoute'
    response = requests.post(url, data=datos)
    ficheroXML = open("infoRuta.xml", 'w')
    ficheroXML.write(response.text)
    ficheroXML.close()

    f= open("infoRuta.xml", "rt")
    arbol=ElementTree.parse(f)
    nodo = arbol.find('ListRouteData')
    nodo2= nodo.find('RouteData')
    nodo3=nodo2.find('DescriptionRouteData')

    fecha = nodo3.find('DescriptionDate')
    horaInicio = nodo3.find('DescriptionInitTime')
    horaFin = nodo3.find('DescriptionEstimateTimeArrival')
    trasbordos = nodo3.find('Transfers')
    duracion = nodo3.find('LongJourney')

    #quitamos lo que no valga en el texto
    fecha=fecha.text.replace("Fecha:","")
    horaInicio=horaInicio.text.replace("Hora de Salida:","")

    print("* Fecha de la ruta: "+fecha)
    print("* Hora de inicio de la ruta: "+horaInicio)
    print("* Hora estimada de llegada: "+horaFin.text)
    print("* Número de trasbordos: "+trasbordos.text)
    print("* Duración del viaje: "+duracion.text)

    print("* Descripcion textual de la ruta:\n")
    listaSecciones=nodo2.find('ListSectionRoute')
    #Recorro todas las secciones
    for seccion in listaSecciones.iter("Section"):
        secciones= list(seccion)
        for paso in secciones:
            if paso.tag == "WalkingLeg":
                leer= paso.find('SourceWalkingLeg')
                leer2=leer.find('RouteDescription')
                #si es lo ultimo que hace quitamos el " hasta"
                andar=leer2.text.replace(" hasta","")
                print("    *"+andar)  
            elif paso.tag == "BusLeg":
                leer= paso.find('SourceBusLeg')
                leer2=leer.find('RouteDescription')
                print("    *"+leer2.text)

#EJERCICIO 4
def calcularRutaMasOptima():
    cOrigen,cDestino = leerCalles()
    xyOrigen=coordenadas(cOrigen)
    xyDestino=coordenadas(cDestino)
    
    caminoOptimo(xyOrigen,xyDestino)
    
#EJERCICIO 5
def puntosDeInteres():
    cOrigen,cDestino = leerCalles()
    xyOrigen=coordenadas(cOrigen)
    xyDestino=coordenadas(cDestino)
    
    pOICaminoOptimo(xyOrigen,xyDestino)
    
def pOICaminoOptimo(xyOrigen,xyDestino):
    datos = {
        'idClient':'WEB.SERV.segonz09@ucm.es',
        'PassKey':'973F19F0-B74A-407C-82E1-37D96CC9940B',
        'statistics':'',
        'cultureInfo':'',
        'coordinateXFrom': xyOrigen[0],
        'coordinateYFrom':xyOrigen[1],
        'originName':'',
        'coordinateXTo': xyDestino[0],
        'coordinateYTo':xyDestino[1],
        'destinationName':'',
        'criteriaSelection':'13',
        'day':'',
        'month':'',
        'year':'',
        'hour':'',
        'minute':'',
        'GenerarAudio':''
    }

    url = 'https://servicios.emtmadrid.es:8443/servicemedia/servicemedia.asmx/GetStreetRoute'
    response = requests.post(url, data=datos)
    ficheroXML = open("infoRuta.xml", 'w')
    ficheroXML.write(response.text)
    ficheroXML.close()

    f= open("infoRuta.xml", "rt")
    arbol=ElementTree.parse(f)
    f.close()
    
    nodo = arbol.find('POIS')
    listaPOIS=nodo.find('ListPOIS')
    
    for poi in listaPOIS.iter("POI"):
        nombre = poi.find('nombre')
        direccion = poi.find('direccion')
        tlfn = poi.find('telefono')
        print("Punto de interés:"+nombre.text)
        
    poiDeseado = input("Punto de interés del que desea información: ")
    for poi in listaPOIS.iter("POI"):
        nombre = poi.find('nombre')
        nom = nombre.text
        if (poiDeseado.lower() == nom.lower()):
            direccion = poi.find('direccion')
            tlfn = poi.find('telefono')
            print("Nombre del punto de interés:"+nom)
            print("Dirección del punto de interés:"+direccion.text)
            print("Teléfono del punto de interés:"+tlfn.text)
            break


Introduzca la calle origen:herencia 2
Introduzca la calle destino:doctor esquerdo
Punto de interés:TAIBESA
Punto de interés:VALLEHERMOSO WAGEN
Punto de interés:KURUMA SPORT
Punto de interés:LISTA MOTOR
Punto de interés:COMERCIAL MERCEDES-BENZ, S.A.
Punto de interés:DEYSA (BRANCH CONDE PEÑALVER)
Punto de interés:TAIBESA
Punto de interés:TALLERES AMADOR, A.P
Punto de interés:TURISMOS MADRID
Punto de interés:TURISMOS MADRID
Punto de interés:KURUMA SPORT
Punto de interés:JOSÉ JURADO
Punto de interés:SERVAUTO
Punto de interés:TALLERES POVEDILLA
Punto de interés:AUTOTODO
Punto de interés:TESTAROSSA CARS
Punto de interés:LISTA MOTOR
Punto de interés:SATE
Punto de interés:BP
Punto de interés:CAMPSA
Punto de interés:CEPSA ELF
Punto de interés:ROLEN MOTOR
Punto de interés:POLCAR
Punto de interés:JOSÉ JURADO
Punto de interés:TALLERES AMADOR, A.P.
Punto de interés:JORBER MOTOR
Punto de interés:DIA, AVENIDA DE LOS TOREROS
Punto de interés:ALCALÁ SUPER, CALLE DE ALCALÁ
Punto de interés:DIA, CALLE DE

Punto de interés del que desea información: pele-mele
Nombre del punto de interés:PELE-MELE
Dirección del punto de interés:CALLE DE JUAN BRAVO
Teléfono del punto de interés:914028001
